# Bayesian Networks in Python

In this tutorial, we will explore how to use Bayesian networks in Python using the `pgmpy` library.

See documentation at:
- https://pgmpy.org/
- https://pgmpy.org/models/bayesiannetwork.html
- https://pgmpy.org/factors/discrete.html
- https://pgmpy.org/exact_infer/ve.html

## Implementation in Python using `pgmpy`.

Let's start by installing the `pgmpy` library.

In [106]:
!pip install -q pgmpy networkx matplotlib plotly daft

Import the required libraries.

In [107]:
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
import networkx as nx
import matplotlib.pyplot as plt
from pgmpy.inference import VariableElimination

### Aufgabe a)

In [108]:
# Define the network structure
model = BayesianNetwork([('TrainStrike', 'JohannaLate'),
                         ('TrainStrike', 'PeterLate'),
                         ('PetersAlarmFails', 'PeterLate')])

statenames = {
'TrainStrike': ['True', 'False'],
'PeterLate': ['True', 'False'],
'JohannaLate': ['True', 'False'],
'PetersAlarmFails': ['True', 'False']}

# Wahrscheinlichkeiten für PetersAlarmFails
cpd_peters_alarm_fails = TabularCPD(variable='PetersAlarmFails', state_names=statenames, variable_card=2, values=[[0.1], [0.9]])
model.add_cpds(cpd_peters_alarm_fails)

# Wahrscheinlichkeiten für den TrainStrike
cpd_train_strike = TabularCPD(variable='TrainStrike', state_names=statenames, variable_card=2, values=[[0.05], [0.95]])
model.add_cpds(cpd_train_strike)

# Wahrscheinlichkeiten für JohannaLate
cpd_johanna_late = TabularCPD(variable='JohannaLate', state_names=statenames, variable_card=2,
                              evidence=['TrainStrike'], evidence_card=[2],
                              values=[
                                  [0.5, 0.02],
                                   [0.5, 0.98]])
model.add_cpds(cpd_johanna_late)


cpd_peter_late = TabularCPD(variable='PeterLate', state_names=statenames, variable_card=2,
                            evidence=['TrainStrike', 'PetersAlarmFails'], evidence_card=[2, 2],
                            values=
                             [[0.7, 0.3, 0.4, 0.01],
                              [0.3, 0.7, 0.6, 0.99]])
model.add_cpds(cpd_peter_late)
model.check_model()

for cpd in model.get_cpds():
  print(cpd)

+-------------------------+-----+
| PetersAlarmFails(True)  | 0.1 |
+-------------------------+-----+
| PetersAlarmFails(False) | 0.9 |
+-------------------------+-----+
+--------------------+------+
| TrainStrike(True)  | 0.05 |
+--------------------+------+
| TrainStrike(False) | 0.95 |
+--------------------+------+
+--------------------+-------------------+--------------------+
| TrainStrike        | TrainStrike(True) | TrainStrike(False) |
+--------------------+-------------------+--------------------+
| JohannaLate(True)  | 0.5               | 0.02               |
+--------------------+-------------------+--------------------+
| JohannaLate(False) | 0.5               | 0.98               |
+--------------------+-------------------+--------------------+
+------------------+-----+-------------------------+
| TrainStrike      | ... | TrainStrike(False)      |
+------------------+-----+-------------------------+
| PetersAlarmFails | ... | PetersAlarmFails(False) |
+------------------+

### Aufgabe b)
Wie hoch ist die Wahrscheinlichkeit, dass Johanna bzw Peter zu spät in die Arbeit kommen?

In [109]:
inference = VariableElimination(model)

print(inference.query(variables=['JohannaLate']))
print(inference.query(variables=['PeterLate']))

+--------------------+--------------------+
| JohannaLate        |   phi(JohannaLate) |
+====================+====================+
| JohannaLate(True)  |             0.0440 |
+--------------------+--------------------+
| JohannaLate(False) |             0.9560 |
+--------------------+--------------------+
+------------------+------------------+
| PeterLate        |   phi(PeterLate) |
+==================+==================+
| PeterLate(True)  |           0.0635 |
+------------------+------------------+
| PeterLate(False) |           0.9365 |
+------------------+------------------+


### Aufgabe c)
Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, falls Peters Wecker
nicht funktioniert?

In [110]:
print(inference.query(['JohannaLate'], evidence={'PetersAlarmFails': 'True'}))

+--------------------+--------------------+
| JohannaLate        |   phi(JohannaLate) |
+====================+====================+
| JohannaLate(True)  |             0.0440 |
+--------------------+--------------------+
| JohannaLate(False) |             0.9560 |
+--------------------+--------------------+


### Aufgabe d)
Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu spät
in die Arbeit kommt?

In [111]:
print(inference.query(['JohannaLate'], evidence={'PeterLate': 'True'}))

+--------------------+--------------------+
| JohannaLate        |   phi(JohannaLate) |
+====================+====================+
| JohannaLate(True)  |             0.1484 |
+--------------------+--------------------+
| JohannaLate(False) |             0.8516 |
+--------------------+--------------------+


### Aufgabe e)
Wie hoch ist die Wahrscheinlichkeit, dass Johanna zu spät in die Arbeit kommt, wenn Peter zu
spät in die Arbeit kommt obwohl Peters Wecker funktioniert?

In [112]:
print(inference.query(['JohannaLate'], evidence={'PeterLate': 'True', 'PetersAlarmFails':'False'}))

+--------------------+--------------------+
| JohannaLate        |   phi(JohannaLate) |
+====================+====================+
| JohannaLate(True)  |             0.3139 |
+--------------------+--------------------+
| JohannaLate(False) |             0.6861 |
+--------------------+--------------------+
